In [1]:
# mnist_cnn_keras
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# mnist 데이터 가져오기
# mnist = tf.keras.datasets.mnist        # MNIST : 손글씨 이미지
mnist = tf.keras.datasets.fashion_mnist  # 의류(Fashion) 이미지
(x_train,y_train),(x_test,y_test) = mnist.load_data()

# X값의 타입을 float형으로 변환
x_train = tf.cast(x_train,dtype=tf.float32)
x_test = tf.cast(x_test,dtype=tf.float32)

# X값의 shape을 4차원으로 변환
x_train = tf.reshape(x_train,[-1,28,28,1])
x_test = tf.reshape(x_test,[-1,28,28,1])
print(x_train.shape)  # (60000, 28, 28, 1)
print(x_test.shape)   # (10000, 28, 28, 1)
print(y_train.shape)  # (60000,)

(60000, 28, 28, 1)
(10000, 28, 28, 1)
(60000,)


In [3]:
# MNIST 분류 CNN keras 모델 구현
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(filters=32,kernel_size=(3,3),padding='same',
                           activation='relu',input_shape=(28,28,1)),
    tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=(2,2),padding='same'),

    tf.keras.layers.Conv2D(filters=64,kernel_size=(3,3),padding='same',
                           activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=(2,2),padding='same'),

    tf.keras.layers.Conv2D(filters=128,kernel_size=(3,3),padding='same',
                           activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=(2,2),padding='same'),

    tf.keras.layers.Flatten(),  
    tf.keras.layers.Dense(units=10,activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
              loss="sparse_categorical_crossentropy",
              metrics=['accuracy'])
model.summary()

# https://yeomko.tistory.com/40
# Xavier Glorot Initialization : W(Weight) 값을 fan_in,fan_out를 사용하여 초기화하여 정확도 향상

# loss 종류
# mean_squared_error : 평균제곱 오차
# binary_crossentropy : 이진분류 오차
# categorical_crossentropy : 다중 분류 오차. one-hot encoding 클래스, [0.2, 0.3, 0.5] 와 같은 출력값과 실측값의 오차값을 계산한다.
# sparse_categorical_crossentropy: 다중 분류 오차. 위와 동일하지만 , integer type 클래스라는 것이 다르다.

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 7, 7, 64)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 7, 7, 128)         73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 4, 4, 128)        0

In [4]:
# 학습
model.fit(x_train,y_train,epochs=25,validation_split=0.25)
# validation_split : 학습데이터를 학습과 평가용 데이터로 분리(75%:25%)

Epoch 1/25
1407/1407 [==============================] - 7s 4ms/step - loss: 0.9777 - accuracy: 0.7744 - val_loss: 0.5001 - val_accuracy: 0.8208
Epoch 2/25
1407/1407 [==============================] - 6s 4ms/step - loss: 0.4989 - accuracy: 0.8170 - val_loss: 0.5068 - val_accuracy: 0.8173
Epoch 3/25
1407/1407 [==============================] - 6s 4ms/step - loss: 0.4922 - accuracy: 0.8172 - val_loss: 0.4962 - val_accuracy: 0.8240
Epoch 4/25
1407/1407 [==============================] - 6s 4ms/step - loss: 0.4740 - accuracy: 0.8248 - val_loss: 0.5568 - val_accuracy: 0.7904
Epoch 5/25
1407/1407 [==============================] - 6s 4ms/step - loss: 0.4696 - accuracy: 0.8258 - val_loss: 0.4620 - val_accuracy: 0.8283
Epoch 6/25
1407/1407 [==============================] - 6s 4ms/step - loss: 0.4599 - accuracy: 0.8277 - val_loss: 0.4851 - val_accuracy: 0.8119
Epoch 7/25
1407/1407 [==============================] - 7s 5ms/step - loss: 0.4528 - accuracy: 0.8304 - val_loss: 0.5395 - val_accuracy:

In [5]:
# train  : 45000(train) + 15000(validatioN)
# batch_size :32   
# 1 epoch : 1407

In [6]:
# 평가
model.evaluate(x_test,y_test)

313/313 [==============================] - 1s 3ms/step - loss: 0.4974 - accuracy: 0.8173


[0.49740859866142273, 0.817300021648407]

In [7]:
# 예측
preds = model.predict(x_test[:10])
print(tf.argmax(preds,axis=1).numpy())
print(y_test[:10])

1/1 [==============================] - 0s 150ms/step
[9 2 1 1 6 1 4 6 5 7]
[9 2 1 1 6 1 4 6 5 7]


### 모델 개선

In [8]:
# VGGNet (VGG-19) 스타일의 MNIST 분류 CNN 모델 
#--------------------------------------------
# ( Conv2D * 2개  --> MaxPool2D ) * 2회 : 4층
# ( Conv2D * 4개  --> MaxPool2D ) * 3회 : 12층
# Dense * 3개                           : 3층
#--------------------------------------------
#                                     총 19층
#--------------------------------------------
# 각 네트워크마다 필터의 수를 2배로 증가 시킨다 : 32-->64-->128-->256-->512

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(filters=32,kernel_size=(3,3),padding='same',
                           activation='relu',input_shape=(28,28,1)),
    tf.keras.layers.Conv2D(filters=64,kernel_size=(3,3),padding='same',
                           activation='relu'),  
    tf.keras.layers.MaxPool2D(pool_size=(2,2)),
    tf.keras.layers.Dropout(rate=0.5),
    
    tf.keras.layers.Conv2D(filters=128,kernel_size=(3,3),padding='same',
                           activation='relu'),
    tf.keras.layers.Conv2D(filters=256,kernel_size=(3,3),padding='valid',
                           activation='relu'),  
    tf.keras.layers.MaxPool2D(pool_size=(2,2)),
    tf.keras.layers.Dropout(rate=0.5),

    tf.keras.layers.Flatten(),  
    tf.keras.layers.Dense(units=512,activation='relu'),
    tf.keras.layers.Dropout(rate=0.5),
    tf.keras.layers.Dense(units=256,activation='relu'),
    tf.keras.layers.Dropout(rate=0.5),
    tf.keras.layers.Dense(units=10,activation='softmax')
])
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 28, 28, 32)        320       
                                                                 
 conv2d_4 (Conv2D)           (None, 28, 28, 64)        18496     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 14, 14, 64)       0         
 2D)                                                             
                                                                 
 dropout (Dropout)           (None, 14, 14, 64)        0         
                                                                 
 conv2d_5 (Conv2D)           (None, 14, 14, 128)       73856     
                                                                 
 conv2d_6 (Conv2D)           (None, 12, 12, 256)       295168    
                                                      

In [9]:
# 학습
model.fit(x_train,y_train,epochs=25,validation_split=0.25)

Epoch 1/25
1407/1407 [==============================] - 13s 9ms/step - loss: 0.9222 - accuracy: 0.7295 - val_loss: 0.4052 - val_accuracy: 0.8459
Epoch 2/25
1407/1407 [==============================] - 12s 9ms/step - loss: 0.4960 - accuracy: 0.8200 - val_loss: 0.3586 - val_accuracy: 0.8674
Epoch 3/25
1407/1407 [==============================] - 12s 8ms/step - loss: 0.4458 - accuracy: 0.8382 - val_loss: 0.3332 - val_accuracy: 0.8690
Epoch 4/25
1407/1407 [==============================] - 12s 8ms/step - loss: 0.4190 - accuracy: 0.8490 - val_loss: 0.3296 - val_accuracy: 0.8769
Epoch 5/25
1407/1407 [==============================] - 12s 9ms/step - loss: 0.4020 - accuracy: 0.8570 - val_loss: 0.3193 - val_accuracy: 0.8798
Epoch 6/25
1407/1407 [==============================] - 11s 8ms/step - loss: 0.3861 - accuracy: 0.8616 - val_loss: 0.2895 - val_accuracy: 0.8848
Epoch 7/25
1407/1407 [==============================] - 11s 8ms/step - loss: 0.3746 - accuracy: 0.8652 - val_loss: 0.2891 - val_ac

In [10]:
# 평가
model.evaluate(x_test,y_test)

313/313 [==============================] - 1s 4ms/step - loss: 0.2917 - accuracy: 0.8928


[0.2917282283306122, 0.892799973487854]